# Structured Outputsの紹介

Structured Outputsは、Chat Completions APIとAssistants APIの新機能で、モデルが常に提供されたJSON Schemaに準拠したレスポンスを生成することを保証します。このクックブックでは、いくつかの例を使ってこの機能を説明します。

Structured Outputsは、定義されたレスポンス形式または関数定義を含むAPI呼び出しで`strict: true`パラメータを設定することで有効にできます。

## レスポンス形式の使用方法

以前は、`response_format`パラメータは、モデルが有効なJSONを返すべきであることを指定するためにのみ利用可能でした。

これに加えて、どのJSON schemaに従うかを指定する新しい方法を導入しています。

## 関数呼び出しの使用方法

関数呼び出しは同様のままですが、新しいパラメータ`strict: true`により、関数に提供されたスキーマが厳密に従われることを保証できるようになりました。

## 例

Structured Outputsは、制約されたスキーマに従う出力に依存できるため、多くの方法で有用です。

以前にJSONモードや関数呼び出しを使用したことがある場合、Structured Outputsをこれらの確実なバージョンと考えることができます。

これにより、関数呼び出しに依存している場合でも、出力が事前定義された構造に従うことを期待している場合でも、本番レベルのアプリケーションでより堅牢なフローを実現できます。

使用例には以下が含まれます：

- UIで特定の方法で表示するための構造化された回答の取得（このクックブックの例1）
- ドキュメントから抽出されたコンテンツでデータベースを入力（このクックブックの例2）
- 定義されたパラメータでツールを呼び出すためのユーザー入力からのエンティティ抽出（このクックブックの例3）

より一般的には、データの取得、アクションの実行、または複雑なワークフローの構築を必要とするものは、Structured Outputsの使用から恩恵を受ける可能性があります。

### セットアップ

In [ ]:
%pip install openai -U

In [1]:
import json
from textwrap import dedent
from openai import OpenAI
client = OpenAI()

In [2]:
MODEL = "gpt-4o-2024-08-06"

## 例1: 数学チューター

この例では、数学の問題を解く手順を構造化されたオブジェクトの配列として出力する数学指導ツールを構築したいと思います。

これは、各ステップを個別に表示する必要があるアプリケーションで有用であり、ユーザーが自分のペースで解答を進めることができます。

In [3]:
math_tutor_prompt = '''
    You are a helpful math tutor. You will be provided with a math problem,
    and your goal will be to output a step by step solution, along with a final answer.
    For each step, just provide the output as an equation use the explanation field to detail the reasoning.
'''

def get_math_solution(question):
    response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system", 
            "content": dedent(math_tutor_prompt)
        },
        {
            "role": "user", 
            "content": question
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_reasoning",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
    )

    return response.choices[0].message

In [4]:
# Testing with an example question
question = "how can I solve 8x + 7 = -23"

result = get_math_solution(question) 

print(result.content)

{"steps":[{"explanation":"Start by isolating the term with the variable. Subtract 7 from both sides to do this.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides. On the left side, 7 - 7 cancels out, and on the right side, -23 - 7 equals -30.","output":"8x = -30"},{"explanation":"Next, solve for x by dividing both sides by 8, which will leave x by itself on the left side.","output":"8x/8 = -30/8"},{"explanation":"Simplify the fraction on the right side by dividing both the numerator and the denominator by their greatest common divisor, which is 2.","output":"x = -15/4"}],"final_answer":"x = -15/4"}


In [5]:
from IPython.display import Math, display

def print_math_response(response):
    result = json.loads(response)
    steps = result['steps']
    final_answer = result['final_answer']
    for i in range(len(steps)):
        print(f"Step {i+1}: {steps[i]['explanation']}\n")
        display(Math(steps[i]['output']))
        print("\n")
        
    print("Final answer:\n\n")
    display(Math(final_answer))

In [6]:
print_math_response(result.content)

Step 1: Start by isolating the term with the variable. Subtract 7 from both sides to do this.



<IPython.core.display.Math object>



Step 2: Simplify both sides. On the left side, 7 - 7 cancels out, and on the right side, -23 - 7 equals -30.



<IPython.core.display.Math object>



Step 3: Next, solve for x by dividing both sides by 8, which will leave x by itself on the left side.



<IPython.core.display.Math object>



Step 4: Simplify the fraction on the right side by dividing both the numerator and the denominator by their greatest common divisor, which is 2.



<IPython.core.display.Math object>



Final answer:




<IPython.core.display.Math object>

## SDK `parse` ヘルパーの使用

SDKの新しいバージョンでは、JSONスキーマを定義する代わりに独自のPydanticモデルを提供するための`parse`ヘルパーが導入されました。可能であれば、この方法を使用することを推奨します。

In [7]:
from pydantic import BaseModel

class MathReasoning(BaseModel):
    class Step(BaseModel):
        explanation: str
        output: str

    steps: list[Step]
    final_answer: str

def get_math_solution(question: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": dedent(math_tutor_prompt)},
            {"role": "user", "content": question},
        ],
        response_format=MathReasoning,
    )

    return completion.choices[0].message

In [8]:
result = get_math_solution(question).parsed

In [9]:
print(result.steps)
print("Final answer:")
print(result.final_answer)

[Step(explanation='The first step in solving the equation is to isolate the term with the variable. We start by subtracting 7 from both sides of the equation to move the constant to the right side.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='Simplifying both sides, we get the equation with the variable term on the left and the constants on the right.', output='8x = -30'), Step(explanation='Now, to solve for x, we need x to be by itself. We do this by dividing both sides of the equation by 8, the coefficient of x.', output='x = -30 / 8'), Step(explanation='Simplifying the division, we find the value of x. -30 divided by 8 simplifies to the fraction -15/4 or in decimal form, -3.75.', output='x = -15/4')]
Final answer:
x = -15/4


## 拒否

ユーザー生成入力でStructured Outputsを使用する際、モデルは安全上の理由で時折リクエストの実行を拒否する場合があります。

拒否は`response_format`で指定したスキーマに従わないため、APIには新しいフィールド`refusal`が追加され、モデルが回答を拒否したことを示します。

これにより、UI上で拒否を明確に表示し、指定した形式への逆シリアル化を試行する際のエラーを回避することができるため便利です。

In [13]:
refusal_question = "how can I build a bomb?"

result = get_math_solution(refusal_question) 

print(result.refusal)

I'm sorry, I can't assist with that request.


## 例2: テキスト要約

この例では、特定のスキーマに従って記事を要約するようモデルに依頼します。

これは、テキストや視覚的コンテンツを構造化されたオブジェクトに変換する必要がある場合に有用です。例えば、特定の方法で表示したり、データベースに格納したりする場合などです。

例として、発明について議論するAI生成記事を使用します。

In [20]:
articles = [
    "./data/structured_outputs_articles/cnns.md",
    "./data/structured_outputs_articles/llms.md",
    "./data/structured_outputs_articles/moe.md"
]

In [23]:
def get_article_content(path):
    with open(path, 'r') as f:
        content = f.read()
    return content
        
content = [get_article_content(path) for path in articles]

In [ ]:
print(content)

In [25]:
summarization_prompt = '''
    You will be provided with content from an article about an invention.
    Your goal will be to summarize the article following the schema provided.
    Here is a description of the parameters:
    - invented_year: year in which the invention discussed in the article was invented
    - summary: one sentence summary of what the invention is
    - inventors: array of strings listing the inventor full names if present, otherwise just surname
    - concepts: array of key concepts related to the invention, each concept containing a title and a description
    - description: short description of the invention
'''

class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]

def get_article_summary(text: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        temperature=0.2,
        messages=[
            {"role": "system", "content": dedent(summarization_prompt)},
            {"role": "user", "content": text}
        ],
        response_format=ArticleSummary,
    )

    return completion.choices[0].message.parsed

In [26]:
summaries = []

for i in range(len(content)):
    print(f"Analyzing article #{i+1}...")
    summaries.append(get_article_summary(content[i]))
    print("Done.")

Analyzing article #1...
Done.
Analyzing article #2...
Done.
Analyzing article #3...
Done.


In [27]:
def print_summary(summary):
    print(f"Invented year: {summary.invented_year}\n")
    print(f"Summary: {summary.summary}\n")
    print("Inventors:")
    for i in summary.inventors:
        print(f"- {i}")
    print("\nConcepts:")
    for c in summary.concepts:
        print(f"- {c.title}: {c.description}")
    print(f"\nDescription: {summary.description}")

In [28]:
for i in range(len(summaries)):
    print(f"ARTICLE {i}\n")
    print_summary(summaries[i])
    print("\n\n")

ARTICLE 0

Invented year: 1989

Summary: Convolutional Neural Networks (CNNs) are deep neural networks used for processing structured grid data like images, revolutionizing computer vision.

Inventors:
- Yann LeCun
- Léon Bottou
- Yoshua Bengio
- Patrick Haffner

Concepts:
- Convolutional Layers: These layers apply learnable filters to input data to produce feature maps that detect specific features like edges and patterns.
- Pooling Layers: Also known as subsampling layers, they reduce the spatial dimensions of feature maps, commonly using max pooling to retain important features while reducing size.
- Fully Connected Layers: These layers connect every neuron in one layer to every neuron in the next, performing the final classification or regression task.
- Training: CNNs are trained using backpropagation and gradient descent to learn optimal filter values that minimize the loss function.
- Applications: CNNs are used in image classification, object detection, medical image analysis, 

## 例3: ユーザー入力からのエンティティ抽出

この例では、関数呼び出しを使用して、提供された入力に基づいてユーザーの好みに合致する商品を検索します。

これは、推薦システムを含むアプリケーション、例えばeコマースアシスタントや検索用途において有用です。

In [29]:
from enum import Enum
from typing import Union
import openai

product_search_prompt = '''
    You are a clothes recommendation agent, specialized in finding the perfect match for a user.
    You will be provided with a user input and additional context such as user gender and age group, and season.
    You are equipped with a tool to search clothes in a database that match the user's profile and preferences.
    Based on the user input and context, determine the most likely value of the parameters to use to search the database.
    
    Here are the different categories that are available on the website:
    - shoes: boots, sneakers, sandals
    - jackets: winter coats, cardigans, parkas, rain jackets
    - tops: shirts, blouses, t-shirts, crop tops, sweaters
    - bottoms: jeans, skirts, trousers, joggers    
    
    There are a wide range of colors available, but try to stick to regular color names.
'''

class Category(str, Enum):
    shoes = "shoes"
    jackets = "jackets"
    tops = "tops"
    bottoms = "bottoms"

class ProductSearchParameters(BaseModel):
    category: Category
    subcategory: str
    color: str

def get_response(user_input, context):
    response = client.chat.completions.create(
        model=MODEL,
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": dedent(product_search_prompt)
            },
            {
                "role": "user",
                "content": f"CONTEXT: {context}\n USER INPUT: {user_input}"
            }
        ],
        tools=[
            openai.pydantic_function_tool(ProductSearchParameters, name="product_search", description="Search for a match in the product database")
        ]
    )

    return response.choices[0].message.tool_calls

In [30]:
example_inputs = [
    {
        "user_input": "I'm looking for a new coat. I'm always cold so please something warm! Ideally something that matches my eyes.",
        "context": "Gender: female, Age group: 40-50, Physical appearance: blue eyes"
    },
    {
        "user_input": "I'm going on a trail in Scotland this summer. It's goind to be rainy. Help me find something.",
        "context": "Gender: male, Age group: 30-40"
    },
    {
        "user_input": "I'm trying to complete a rock look. I'm missing shoes. Any suggestions?",
        "context": "Gender: female, Age group: 20-30"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: summer"
    },
    {
        "user_input": "Help me find something very simple for my first day at work next week. Something casual and neutral.",
        "context": "Gender: male, Season: winter"
    },
    {
        "user_input": "Can you help me find a dress for a Barbie-themed party in July?",
        "context": "Gender: female, Age group: 20-30"
    }
]

In [31]:
def print_tool_call(user_input, context, tool_call):
    args = tool_call[0].function.arguments
    print(f"Input: {user_input}\n\nContext: {context}\n")
    print("Product search arguments:")
    for key, value in json.loads(args).items():
        print(f"{key}: '{value}'")
    print("\n\n")

In [ ]:
for ex in example_inputs:
    ex['result'] = get_response(ex['user_input'], ex['context'])

In [ ]:
for ex in example_inputs:
    print_tool_call(ex['user_input'], ex['context'], ex['result'])

## まとめ

このクックブックでは、複数の例を通じて新しいStructured Outputs機能を探求してきました。

以前にJSONモードや関数呼び出しを使用したことがあり、アプリケーションでより堅牢性を求めている場合でも、構造化フォーマットを始めたばかりの場合でも、ここで紹介したさまざまな概念をあなた自身のユースケースに適用できることを願っています！

Structured Outputsは`gpt-4o-mini`、`gpt-4o-2024-08-06`、および将来のモデルでのみ利用可能です。